In [1]:
import os 
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from itertools import product

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score



In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load CIFAR-10 dataset

The CIFAR-10 dataset consists of images that are 32x32 pixels in size and have 3 color channels (Red, Green, Blue)  
Each image in the CIFAR-10 dataset is represented as a 1D array of length 3072 (32 * 32 * 3).
The first 1024 values correspond to the Red channel, the next 1024 values correspond to the Green channel, and the last 1024 values correspond to the Blue channel.

The common format for image data in deep learning frameworks like PyTorch is:

Shape: (num_samples, channels, height, width)

In [ ]:
# Initialize variables
X_train, y_train = [], []
X_test, y_test = [], []

# Load all the paths of the pickle files
cifar_path = "CIFAR-10"
files_path = os.listdir(cifar_path)

# Load training data
for file in files_path: 
  filepath = os.path.join(cifar_path, file)
  if file.startswith("data_batch"):
    temp_dict = unpickle(filepath)
    X_train.extend(temp_dict[b'data'])
    y_train.extend(temp_dict[b'labels'])
    print(len(X_train), len(y_train))

# Load testing data
for file in files_path:
    filepath = os.path.join(cifar_path, file)
    if file.startswith("test_batch"):
        temp_dict = unpickle(filepath)
        X_test.append(temp_dict[b'data'])
        y_test.extend(temp_dict[b'labels'])


# Scale the data
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

# Turn into numpy array 
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the data
# X_train = np.vstack(X_train).reshape(-1, 3, 32, 32) #-1 is the number of samples/images, 3 is the channnels, 32 is the height and 32 is the width
X_train = np.vstack(X_train)

# X_test = np.vstack(X_test).reshape(-1, 3, 32, 32)
X_test= np.vstack(X_test)
print(X_train.shape, y_train.shape)

10000 10000
20000 20000
30000 30000
40000 40000
50000 50000
(50000, 3072) (50000,)


In [4]:
# Create X_val set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=13)
X_train.shape, X_val.shape

((40000, 3072), (10000, 3072))

In [5]:
# import matplotlib.pyplot as plt

# # Select an image index
# index = 13

# # Get the image
# image = X_train[index]

# # Plot it
# plt.figure(figsize=(1, 1))  
# plt.imshow(image.reshape(0, 2, 3, 1))
# plt.axis("off")
# plt.show()


For PyTorch: Image data in the format (num_samples, channels, height, width) for training and inference.  
For Visualization: transpose(1, 2, 0) is required to convert the image to the format (height, width, channels) when using plt.imshow().

In [6]:
n_hidden_layers = [1, 2, 4]
n_neurons_x_layer = [50, 200, 1000]
learning_rate = [10**-3, 10**-4, 10**-5]
activation = 'relu'
solver='adam'

# se hacen todas las combinaciones
configurations = list(product(n_hidden_layers, n_neurons_x_layer, learning_rate))

configuration_results = {
    'hidden_layers': [],
    'n_neurons': [],
    'learning_rate': [],
    'train_time': [],
    'train_score': [],
    'val_score': [],  # stores only the last validation score after each epoch? 
    'test_score': [],  # stores only the last validation score after each epoch? 
    'confusion_matrices': [],
    'val_accuracy_over_epochs': [] #store learning curves for all configurations
}



In [ ]:
for i, (h, n, lr) in enumerate(configurations):
    print(i + 1, '/', len(configurations))
    print('Hidden Layers: {}, # Neurons: {}, Learning rate: {}'.format(h, n, lr))
    # definir estructura de neurona
    neuron_structure = (np.ones(h) * n).astype(int)

    # Entrenar NN
    mlp = MLPClassifier(
        hidden_layer_sizes=(neuron_structure),
        activation='relu',
        solver='adam',
        learning_rate_init=lr
    )

    # Start timing
    start_time = time.time()
    
    # Train for one epoch
    mlp.partial_fit(X_train, y_train, classes=np.unique(y_train))

    # Calculate time taken
    total_train_time = time.time() - start_time
    # Calculate accuracies
    train_accuracy = mlp.score(X_train, y_train)
    val_accuracy = mlp.score(X_val, y_val)
    y_val_pred = mlp.predict(X_val)  # Predict once

    # Store validation accuracy
    val_accuracy_over_epochs = [accuracy_score(y_val, y_val_pred)]

    # Confusion matrix
    best_cm = confusion_matrix(y_val, y_val_pred)

    # Calculate test accuracy
    test_accuracy = mlp.score(X_test, y_test)

    # Print results
    print(f"Training Time: {total_train_time:.4f} seconds")
    # print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    # print(f"Test Accuracy: {test_accuracy:.4f}")
    # print("Confusion Matrix:\n", best_cm)

    # Se almacenan los resultados en el dict
    configuration_results['hidden_layers'].append(h)
    configuration_results['n_neurons'].append(n)
    configuration_results['learning_rate'].append(lr)
    configuration_results['train_time'].append(total_train_time)
    configuration_results['train_score'].append(train_accuracy)
    configuration_results['val_score'].append(val_accuracy)
    configuration_results['test_score'].append(test_accuracy)
    configuration_results['confusion_matrices'].append(best_cm)
    configuration_results['val_accuracy_over_epochs'].append(val_accuracy_over_epochs)

configurations_df = pd.DataFrame(configuration_results)
configurations_df.to_excel('One_Epoch_Results.xlsx', index=False)

1 / 27
Hidden Layers: 1, # Neurons: 50, Learning rate: 0.001
Training Time: 1.2941 seconds
Validation Accuracy: 0.1006
2 / 27
Hidden Layers: 1, # Neurons: 50, Learning rate: 0.0001
Training Time: 1.2607 seconds
Validation Accuracy: 0.1061
3 / 27
Hidden Layers: 1, # Neurons: 50, Learning rate: 1e-05
Training Time: 1.2636 seconds
Validation Accuracy: 0.1170
4 / 27
Hidden Layers: 1, # Neurons: 200, Learning rate: 0.001
Training Time: 3.2648 seconds
Validation Accuracy: 0.2283
5 / 27
Hidden Layers: 1, # Neurons: 200, Learning rate: 0.0001
Training Time: 3.2385 seconds
Validation Accuracy: 0.2332
6 / 27
Hidden Layers: 1, # Neurons: 200, Learning rate: 1e-05
Training Time: 3.2637 seconds
Validation Accuracy: 0.1957
7 / 27
Hidden Layers: 1, # Neurons: 1000, Learning rate: 0.001
Training Time: 14.9456 seconds
Validation Accuracy: 0.2136
8 / 27
Hidden Layers: 1, # Neurons: 1000, Learning rate: 0.0001
Training Time: 15.2427 seconds
Validation Accuracy: 0.2604
9 / 27
Hidden Layers: 1, # Neurons: 

ModuleNotFoundError: No module named 'openpyxl'

In [8]:
configurations_df.to_excel('One_Epoch_Results.xlsx', index=False)

In [9]:
configurations_df.sort_values(by='val_score', ascending=False)

,hidden_layers,n_neurons,learning_rate,train_time,train_score,val_score,test_score,confusion_matrices,val_accuracy_over_epochs
24,4,1000,0.00100,30.618588,0.350600,0.3471,0.3467,"[[476, 25, 45, 54, 33, 16, 42, 15, 199, 81], [...",[0.3471]
25,4,1000,0.00010,30.914072,0.322025,0.2949,0.3119,"[[269, 67, 247, 51, 30, 35, 22, 89, 105, 71], ...",[0.2949]
16,2,1000,0.00010,20.982982,0.289650,0.2772,0.2791,"[[407, 10, 48, 150, 102, 11, 13, 41, 121, 83],...",[0.2772]
26,4,1000,0.00001,30.993947,0.287600,0.2757,0.2741,"[[404, 53, 35, 61, 81, 38, 16, 96, 153, 49], [...",[0.2757]
21,4,200,0.00100,3.629891,0.266900,0.2661,0.2598,"[[246, 66, 106, 35, 130, 49, 26, 12, 191, 125]...",[0.2661]
7,1,1000,0.00010,15.242721,0.269100,0.2604,0.2564,"[[0, 50, 181, 43, 2, 2, 18, 50, 555, 85], [1, ...",[0.2604]
17,2,1000,0.00001,20.558359,0.266575,0.2587,0.2577,"[[230, 58, 71, 42, 52, 73, 15, 68, 331, 46], [...",[0.2587]
15,2,1000,0.00100,20.638288,0.242550,0.2462,0.2438,"[[370, 161, 56, 40, 23, 63, 4, 26, 212, 31], [...",[0.2462]
8,1,1000,0.00001,15.333848,0.241875,0.2386,0.2293,"[[273, 53, 100, 73, 29, 46, 26, 39, 228, 119],...",[0.2386]
4,1,200,0.00010,3.238498,0.234675,0.2332,0.2316,"[[274, 57, 188, 20, 28, 44, 9, 42, 144, 180], ...",[0.2332]


In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32) / 255.0  # Normalize to [0, 1]
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32) / 255.0  # Normalize to [0, 1]
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for training and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
class SimpleDenseNN(nn.Module):
    def __init__(self, num_layers, neurons_per_layer):
        super(SimpleDenseNN, self).__init__()
        
        # Create a list to hold the layers
        layers = []
        
        # Input size for the first layer
        input_size = 3 * 32 * 32  # Assuming input images are flattened (3 channels, 32x32 pixels)
        
        # Create the specified number of layers
        for i in range(num_layers):
            layers.append(nn.Linear(input_size, neurons_per_layer))  # Add a dense layer
            layers.append(nn.ReLU())  # Add ReLU activation
            input_size = neurons_per_layer  # Update input size for the next layer
        
        # Add the output layer
        layers.append(nn.Linear(neurons_per_layer, 10))  # Assuming 10 output classes for CIFAR-10
        
        # Combine all layers into a sequential model
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input tensor
        return self.model(x)  # Forward pass through the model

# Example usage
num_layers = 3  # Desired number of layers
neurons_per_layer = 128  # Desired number of neurons per layer
model = SimpleDenseNN(num_layers, neurons_per_layer)

# Print the model architecture
# print(model)

# Create an instance of the model
model = SimpleDenseNN(num_layers=3, neurons_per_layer=2)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 5  # Number of epochs to train
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()   # Zero the parameter gradients
        outputs = model(inputs) # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()         # Backward pass
        optimizer.step()        # Optimize the weights
        running_loss += loss.item()
        
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Save the model (optional)
torch.save(model.state_dict(), 'simple_cnn.pth')

# Prediction on the test set
correct = 0
total = 0
with torch.no_grad():  # No need to track gradients during evaluation
    for data in test_loader:
        images, labels = data
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        total += labels.size(0)  # Update total count
        correct += (predicted == labels).sum().item()  # Update correct count

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')

# Print predicted labels
outputs = model(images)
_, predicted = torch.max(outputs, 1)
print('Predicted labels:', predicted.numpy())
print('True labels:', labels.numpy())

Epoch 1, Batch 100, Loss: 2.409
Epoch 1, Batch 200, Loss: 2.380
Epoch 1, Batch 300, Loss: 2.365
Epoch 1, Batch 400, Loss: 2.352
Epoch 1, Batch 500, Loss: 2.341
Epoch 1, Batch 600, Loss: 2.340
Epoch 1, Batch 700, Loss: 2.331
Epoch 2, Batch 100, Loss: 2.323
Epoch 2, Batch 200, Loss: 2.318
Epoch 2, Batch 300, Loss: 2.315
Epoch 2, Batch 400, Loss: 2.312
Epoch 2, Batch 500, Loss: 2.315
Epoch 2, Batch 600, Loss: 2.308
Epoch 2, Batch 700, Loss: 2.309
Epoch 3, Batch 100, Loss: 2.307
Epoch 3, Batch 200, Loss: 2.306
Epoch 3, Batch 300, Loss: 2.306
Epoch 3, Batch 400, Loss: 2.305
Epoch 3, Batch 500, Loss: 2.304
Epoch 3, Batch 600, Loss: 2.304
Epoch 3, Batch 700, Loss: 2.304
Epoch 4, Batch 100, Loss: 2.304
Epoch 4, Batch 200, Loss: 2.304
Epoch 4, Batch 300, Loss: 2.303
Epoch 4, Batch 400, Loss: 2.303
Epoch 4, Batch 500, Loss: 2.303
Epoch 4, Batch 600, Loss: 2.303
Epoch 4, Batch 700, Loss: 2.302
Epoch 5, Batch 100, Loss: 2.302
Epoch 5, Batch 200, Loss: 2.303
Epoch 5, Batch 300, Loss: 2.302
Epoch 5,

In [ ]:

# Define the number of layers and neurons per layer
num_layers = 3  # Number of layers
neurons_per_layer = 2  # Number of neurons in each layer

# Create a list to hold the layers
layers = []

# Input size for the first layer
input_size = 2

# Create layers in a loop
for _ in range(num_layers):
    layer = nn.Linear(input_size, neurons_per_layer)
    layers.append(layer)
    # Update input_size for the next layer
    input_size = neurons_per_layer